In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "7"
from kaijsra.simulation import bpg_ldpc_transmission
from kaijsra.utils.channel_coding.ldpc import LDPC_modulation_codec
import numpy as np
from kaijsra import dsetroot
import pickle

In [11]:
dataset_path = dsetroot+"Kodak/"
image_paths = [dataset_path+ image for image in os.listdir(dataset_path)]
modulation_bps = 2
snr_db = 10
compression_values = list(range(35,50,2))
compression_values = [50]
k = int(4096*3/4)
n = 4096
ldpc_transceiver = LDPC_modulation_codec(k,n,modulation_bps,snr_db)
results_over_compression_value = {
    "ldpc_k":k,
    "ldpc_n":n,
    "snr_db":snr_db,
    "modulation_bps":modulation_bps,
    "dataset_path":dataset_path,
    "compression_values":compression_values
}
# # sn.config.xla_compat=True
print("finished init")
for compression_value in compression_values:
    eval_results = bpg_ldpc_transmission(image_paths,
                                        compression_value,
                                        ldpc_transceiver,
                                        run_mode="eager",
                                        process_num=30)
    results_over_compression_value[compression_value] = eval_results
# # sn.config.xla_compat=False

finished init


100%|██████████| 1/1 [00:51<00:00, 51.69s/it]


In [4]:
avg_psnrs = []
avg_bwrs = []
for compression_value in compression_values:
    avg_psnrs.append(np.mean(results_over_compression_value[compression_value]['psnrs']))
    avg_bwrs.append(np.mean(results_over_compression_value[compression_value]['bwrs']))
    print(np.mean(results_over_compression_value[compression_value]['psnrs']),
          np.mean(results_over_compression_value[compression_value]['bwrs']))

25.60776524994634 0.012502788025655865


In [5]:
with open('../evalresults/CLIC_ldpc_fix_snr_2_3.pkl','wb') as f:
    pickle.dump(results_over_compression_value,f)


# Code segment for loading the pickle saved data

In [1]:
import pickle
import numpy as np
with open("../evalresults/CLIC_comp_37_ldpc_fix_snr_0.75_mod_2_chn_10.pkl",'rb') as f:
    results = pickle.load(f)
# results
snr_dbs = results["snr_dbs"]
for snr_db in snr_dbs:
    print(snr_db,np.mean(results[snr_db]["bwrs"]),
          np.mean(results[snr_db]["psnrs"]))
# results


4.5 0.06389170877098159 25.86843830392812
4.5625 0.06389170877098159 30.142838305066967
4.59375 0.06389170877098159 31.740993634795668
4.625 0.06389170877098159 32.686006051516394
4.6875 0.06389170877098159 33.712794561973915
4.75 0.06389170877098159 34.11437579642389
6 0.06389170877098159 34.32150842742132
8 0.06389170877098159 34.32150842742132
12 0.06389170877098159 34.32150842742132


In [3]:
import pickle
import numpy as np
with open("../evalresults/Kodak_ldpc_fix_snr.pkl",'rb') as f:
    results = pickle.load(f)
# results
compression_values = results["compression_values"]
for compression_value in compression_values:
    print(compression_value,np.mean(results[compression_value]["bwrs"]),
          np.mean(results[compression_value]["psnrs"]))
print(results['ldpc_k'])

35 0.1407704178537512 33.601639557650714
37 0.10938390313390313 32.36920797083861
39 0.08414033882783883 31.200139239144505
41 0.06238001967168633 30.01488457319007
43 0.046241181657848324 28.953755458646864
45 0.03394552977886311 27.950165073945886
47 0.024108626712793377 26.95397199539815
49 0.017328254307420975 26.05901565791669
2730


# Code to process saved results

In [53]:
import pickle
import os
import pandas as pd
from collections import defaultdict
# first process PSNR-BWR data
with open("../evalresults/CLIC_ldpc_fix_snr_3_4.pkl","rb") as f:
    raw_data = pickle.load(f)
# first obtain the compression indices
compression_indices = raw_data["compression_values"]
# use some lists to collect the target information
data = defaultdict(list)
data["BPG compression index"] = compression_indices
for compression_index in compression_indices:
    average_bwr = np.mean(
        raw_data[compression_index]["bwrs"])
    data["bandwidth ratio"].append(average_bwr)
    average_psnr = np.mean(
        raw_data[compression_index]["psnrs"])
    data["psnr"].append(average_psnr)
    average_ssim = np.mean(
        raw_data[compression_index]["ssims"])
    data["ms-ssim"].append(average_ssim)
dataframe = pd.DataFrame(data)
dataset_name = raw_data["dataset_path"].split(os.sep)[-2]
information_column = [
    ["dataset", dataset_name],
    ["ldpc_k", raw_data["ldpc_k"]],
    ["ldpc_n", raw_data["ldpc_n"]],
    ["ldpc code rate", raw_data["ldpc_k"]/raw_data["ldpc_n"]],
    ["snr_db", raw_data["snr_db"]],
    ["modulation bits/symbol", raw_data["modulation_bps"]]
]
information_df = pd.DataFrame(information_column,
                              columns=["parameter name", "parameter value"])
save_df = pd.concat([dataframe,information_df],axis=1)
save_df

,BPG compression index,bandwidth ratio,psnr,ms-ssim,parameter name,parameter value
0,35,0.082683,35.430256,0.979063,dataset,CLIC
1,37,0.063892,34.321508,0.973616,ldpc_k,3072
2,39,0.049411,33.260584,0.967251,ldpc_n,4096
3,41,0.037194,32.148455,0.958789,ldpc code rate,0.75
4,43,0.028167,31.145521,0.949484,snr_db,10
5,45,0.021195,30.162957,0.938069,modulation bits/symbol,2
6,47,0.015495,29.157192,0.923845,NaN,NaN
7,49,0.011428,28.235662,0.907374,NaN,NaN


In [54]:
save_df.to_excel(
    f"../evalresults/{dataset_name}_psnr_vs_bwr.xlsx")

In [57]:
import pickle
import os
import pandas as pd
from collections import defaultdict
# now process PSNR-SNR data
with open("../evalresults/CLIC_comp_37_ldpc_fix_snr_0.75_mod_2_chn_10.pkl","rb") as f:
    raw_data = pickle.load(f)
# first obtain the compression indices
snr_dbs = raw_data["snr_dbs"]
# use some lists to collect the target information
data = defaultdict(list)
data["snr_db"] = snr_dbs
for snr_db in snr_dbs:
    average_bwr = np.mean(raw_data[snr_db]["bwrs"])
    data["bandwidth ratio"].append(average_bwr)
    average_psnr = np.mean( raw_data[snr_db]["psnrs"])
    data["psnr"].append(average_psnr)
    average_ssim = np.mean(raw_data[snr_db]["ssims"])
    data["ms-ssim"].append(average_ssim)
dataframe = pd.DataFrame(data)
dataset_name = raw_data["dataset_path"].split(os.sep)[-2]
information_column = [
    ["dataset", dataset_name],
    ["ldpc_k", raw_data["ldpc_k"]],
    ["ldpc_n", raw_data["ldpc_n"]],
    ["ldpc code rate", raw_data["ldpc_k"]/raw_data["ldpc_n"]],
    ["BPG compression index", raw_data["compression_value"]],
    ["modulation bits/symbol", raw_data["modulation_bps"]]
]
information_df = pd.DataFrame(information_column,
                              columns=["parameter name", "parameter value"])
save_df = pd.concat([dataframe,information_df],axis=1)
save_df

,snr_db,bandwidth ratio,psnr,ms-ssim,parameter name,parameter value
0,4.50000,0.063892,25.868438,0.843258,dataset,CLIC
1,4.56250,0.063892,30.142838,0.915562,ldpc_k,3072
2,4.59375,0.063892,31.740994,0.936565,ldpc_n,4096
3,4.62500,0.063892,32.686006,0.948681,ldpc code rate,0.75
4,4.68750,0.063892,33.712795,0.965606,BPG compression index,37
5,4.75000,0.063892,34.114376,0.970332,modulation bits/symbol,2
6,6.00000,0.063892,34.321508,0.973616,NaN,NaN
7,8.00000,0.063892,34.321508,0.973616,NaN,NaN
8,12.00000,0.063892,34.321508,0.973616,NaN,NaN


In [58]:
save_df.to_excel(
    f"../evalresults/{dataset_name}_psnr_vs_snr.xlsx")